# Compute Zonal Means


In [ ]:
%load_ext dotenv
%dotenv

import os
from importlib import reload
from typing import cast

import pandas as pd

from cholera import ceda, lst, outbreaks, precip, xarray_ as xr_

# Reload modules to avoid the need to restart the notebook kernel after making
# code changes within the modules.
reload(lst)
reload(precip)
reload(xr_)
None

In [ ]:
def zonal_means_to_csv(zonal_means: pd.DataFrame) -> str:
    times = zonal_means.index.get_level_values("time")
    begin_time = cast(pd.Timestamp, times.min()).date()
    end_time = cast(pd.Timestamp, times.max()).date()
    path = (
        f"../data/zonal-means-{zonal_means.columns[0]}-"
        f"{begin_time.strftime('%Y%m')}-{end_time.strftime('%Y%m')}.csv"
    )
    zonal_means.to_csv(path)

    return path

In [ ]:
expiry = ceda.auth(
    username=os.environ["CEDA_USERNAME"],
    password=os.environ["CEDA_PASSWORD"],
)

print(f"CEDA OPeNDAP certificate is valid through {expiry.isoformat()} UTC")

In [ ]:
regions = outbreaks.regions()
start_months = outbreaks.start_month_range()

# Entire monthly range of outbreaks, with a 3-month prelude
all_months = slice(start_months[0] - pd.offsets.MonthBegin(3), start_months[-1])
print(all_months)

In [ ]:
lst_zonal_means = lst.zonal_means(regions, time=all_months)
zonal_means_to_csv(lst_zonal_means)
lst_zonal_means

In [ ]:
precip_zonal_means = precip.zonal_means(regions, time=all_months)
zonal_means_to_csv(precip_zonal_means)
precip_zonal_means